# Data Preprocessing

In [69]:
import numpy as np
import os
import cv2
import tensorflow as tf
import pickle
from random import shuffle
import matplotlib.pyplot as plt
import datetime

In [70]:
dataset = os.path.join(os.getcwd(), "pdataset")
data = []
for i , value in enumerate(os.listdir(dataset)):
    print(value)
    f_dir = os.path.join(dataset,value)
    for each in os.listdir(f_dir):
        image = cv2.imread(os.path.join(f_dir,each),0)
        image = cv2.resize(image,(200,200))
        data.append((image,i))

normal
stroke


In [71]:
shuffle(data)
x = []
y = []
for features , label in data:
    x.append(features)
    y.append(label)
x = np.array(x).reshape(-1,200,200,1)
pickle_out = open("X.pickle","wb")
pickle.dump(x,pickle_out)
pickle_out.close()

pickle_out = open("Y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

# Training

In [72]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
import pickle

In [79]:
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

X = X/255.0

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  

model.add(Dense(64))

model.add(Dense(1,activation='sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

logdir = "logs\\scalars\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
model.fit(X, y, batch_size=32, epochs=15, validation_split=0.3,  callbacks=[tensorboard_callback])

model.save("model.h5")

Train on 540 samples, validate on 232 samples
Epoch 1/15
540/540 [==============================] - 3s 5ms/sample - loss: 0.6837 - acc: 0.5889 - val_loss: 0.6536 - val_acc: 0.6336
Epoch 2/15
540/540 [==============================] - 2s 4ms/sample - loss: 0.6586 - acc: 0.6352 - val_loss: 0.6588 - val_acc: 0.6466
Epoch 3/15
540/540 [==============================] - 2s 3ms/sample - loss: 0.6459 - acc: 0.6426 - val_loss: 0.6392 - val_acc: 0.6336
Epoch 4/15
540/540 [==============================] - 2s 3ms/sample - loss: 0.6204 - acc: 0.6519 - val_loss: 0.6095 - val_acc: 0.6552
Epoch 5/15
540/540 [==============================] - 2s 3ms/sample - loss: 0.5653 - acc: 0.6889 - val_loss: 0.5834 - val_acc: 0.6810
Epoch 6/15
540/540 [==============================] - 2s 3ms/sample - loss: 0.4942 - acc: 0.7370 - val_loss: 0.5334 - val_acc: 0.7026
Epoch 7/15
540/540 [==============================] - 2s 3ms/sample - loss: 0.4311 - acc: 0.7889 - val_loss: 0.5269 - val_acc: 0.7500
Epoch 8/15
540/5

# Testing

In [74]:
model = tf.keras.models.load_model('model.h5')
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_56 (Conv2D)           (None, 198, 198, 32)      320       
_________________________________________________________________
activation_65 (Activation)   (None, 198, 198, 32)      0         
_________________________________________________________________
max_pooling2d_56 (MaxPooling (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 97, 97, 32)        9248      
_________________________________________________________________
activation_66 (Activation)   (None, 97, 97, 32)        0         
_________________________________________________________________
max_pooling2d_57 (MaxPooling (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 46, 46, 64)      

In [75]:
from matplotlib import pyplot as plt
%matplotlib inline

In [76]:
def ptest(path):
    image = cv2.imread(path,0)
    image = cv2.resize(image,(200,200))
    x = np.array(image).reshape(-1,200,200,1)
    y = model.predict_classes(x)
    return y[0][0]

In [80]:
for i , each in enumerate(os.listdir("ptest")): #[ "normal" = 0,"stroke" = 1]
    print(each,f"expected {i} : observerd ->")

    for file in os.listdir(f"ptest/{each}"):
        print(file," -> ",ptest(f"ptest/{each}/"+file))
        

normal expected 0 : observerd ->
26.jpg  ->  0
32.jpg  ->  0
70.jpg  ->  0
stroke expected 1 : observerd ->
103.jpg  ->  1
148.jpg  ->  0
249.jpg  ->  0
253.jpg  ->  1
278.jpg  ->  1
